In [1]:
import numpy as np 
import pandas as pd 
from statistics import mean
import math
import requests
import json 
from datetime import datetime,timedelta
import os
import math
import sys
from scipy.optimize import minimize_scalar

In [ ]:
# File paths to merge for given month
files = [
    'MOD-PM-00562-ID-CLASSROOM.csv' 
]

# Classroom numbers for merging
classrooms=  ['clasroom numbers']

# Classroom Ids for merging. There are 50 classrooms. 38 from one school and 12 from other school.
classroomIds = [i for i in range(1,39)]

new_df = pd.DataFrame(columns=['Timestamp', 'PM1','PM25','PM10','sample_rh','sample_temp','Id','classroom'])

for f in range(len(files)):
    file = pd.read_csv(files[f])
    file.insert(6, "Id", classroomIds[f])
    file.insert(7, "classroom", classrooms[f])
    new_df = pd.concat([new_df,file])

# Saving the file 
new_df.to_csv('Month_Name.csv',index=False)

In [ ]:
# This function takes csv file as input which genertaed in above code and calculate monthly average for all classroom IDs.
def monthly_avg(filePath):
    file = pd.read_csv(filePath)
    classDictPm25 = {}
    classDictPm1 = {}
    classDictPm10 = {}
    for ind in file.index:
        key = file['Id'][ind]

        pm25List = classDictPm25.get(key,[])
        pm25List.append(file['PM25'][ind])
        classDictPm25[key] = pm25List

        pm1List = classDictPm1.get(key,[])
        pm1List.append(file['PM1'][ind])
        classDictPm1[key] = pm1List

        pm10List = classDictPm10.get(key,[])
        pm10List.append(file['PM10'][ind])
        classDictPm10[key] = pm10List
    for k in classDictPm25:
        print(k)
    print('************************** PM 1 Averages **************************')
    for k in classDictPm1:
        m = np.nanmean(classDictPm1[k])
        if math.isnan(m):
            print(k)
            print(classDictPm1[k])
        print(m)
 
    print('************************** PM 25 Averages **************************')
    for k in classDictPm25:
        m = np.nanmean(classDictPm25[k])
        if math.isnan(m):
            print(k)
            print(classDictPm25[k])
        print(m)
    print('************************** PM 10 Averages **************************')
    for k in classDictPm10:
        m = np.nanmean(classDictPm10[k])
        if math.isnan(m):
            print(k)
            print(classDictPm10[k])
        print(m)



In [ ]:
# This function takes csv file as input, it will generate daily average of classroom based on date.
def daily_avg(filePath):
    file = pd.read_csv(filePath)
    classDictPm25 = {}
    classDictPm1 = {}
    classDictPm10 = {}
    for ind in file.index:
        key = datetime.datetime.strptime(file['Timestamp'][ind],"%Y-%m-%dT%H:%M:%SZ").date()

        pm25List = classDictPm25.get(key,[])
        pm25List.append(file['PM25'][ind])
        classDictPm25[key] = pm25List

        pm1List = classDictPm1.get(key,[])
        pm1List.append(file['PM1'][ind])
        classDictPm1[key] = pm1List

        pm10List = classDictPm10.get(key,[])
        pm10List.append(file['PM10'][ind])
        classDictPm10[key] = pm10List
    for k in classDictPm1:
        print(k)
    print('************************** PM 1 Averages **************************')
    pm1 = []
    pm25 = []
    pm10 = []
    for k in classDictPm1:
        m = np.nanmean(classDictPm1[k])
        if math.isnan(m):
            print(k)
            print(classDictPm1[k])
        print(m)
 
    print('************************** PM 25 Averages **************************')
    for k in classDictPm25:
        m = np.nanmean(classDictPm25[k])
        if math.isnan(m):
            print(k)
            print(classDictPm25[k])
        print(m)
        
    print('************************** PM 10 Averages **************************')
    for k in classDictPm10:
        m = np.nanmean(classDictPm10[k])
        if math.isnan(m):
            print(k)
            print(classDictPm10[k])
        print(m)

    return pm1,pm25,pm10

In [ ]:
# This function takes csv file path to read data, dates which needs to be removed from the data and save path to where new cleaned data needs to be saved. 
def remove_dates(filePath,dates, savePath):
    file = pd.read_csv(filePath)
    
    pm1List = []
    pm25List = []
    pm10List = []
    timestampList = []
    sample_temp = []
    sample_rh = []
    
    for ind in file.index:
            currentDate = datetime.datetime.strptime(file['Timestamp'][ind],"%Y-%m-%dT%H:%M:%S").date()
            if currentDate not in dates:
                timestampList.append(file['Timestamp'][ind])
                sample_rh.append(file['sample_rh'][ind])
                sample_temp.append(file['sample_temp'][ind])
                pm1List.append(file['PM1'][ind])
                pm25List.append(file['PM25'][ind])
                pm10List.append(file['PM10'][ind])
                
    newFile = pd.DataFrame({'Timestamp':timestampList,
                            'PM1' : pm1List,
                            'PM25': pm25List,
                            'PM10': pm10List,
                            'sample_rh' :sample_rh,
                            'sample_temp': sample_temp})
    newFile.to_csv(savePath,index=False)

In [ ]:
# This function takes CSV path to save data, cleans the data and save the data to save path.
def clean_data(path,savePath):
    df = pd.read_csv(path)
    sv = pd.DataFrame(columns=['Timestamp', 'PM1','PM25','PM10','sample_rh','sample_temp'])
        
   
    for ind in df.index:
        dateTime = datetime.strptime(df['Timestamp'][ind],"%Y-%m-%dT%H:%M:%S")
        
        if dateTime.weekday() == 4:
            date = dateTime.replace(hour=14, minute=0, second=0)
            if dateTime <= date: 
                sv.loc[len(sv)] = {'Timestamp': df['Timestamp'][ind], 'PM1': df['PM1'][ind], 'PM25': df['PM25'][ind],
                                   'PM10': df['PM10'][ind],'sample_rh': df['sample_rh'][ind], 'sample_temp': df['sample_temp'][ind]}
                
        else:
            date = dateTime.replace(hour=15, minute=0, second=0)
            if dateTime <= date: 
                sv.loc[len(sv)] = {'Timestamp': df['Timestamp'][ind], 'PM1': df['PM1'][ind], 'PM25': df['PM25'][ind],
                                   'PM10': df['PM10'][ind],'sample_rh': df['sample_rh'][ind], 'sample_temp': df['sample_temp'][ind]}

    sv.to_csv(savePath,index=False) 
            

In [ ]:
# This function is used to calculate the ACH value from CO2 data.
def CO2_ACH(path):
    X = 0
    df = pd.read_csv(path)
    df['time'] = range(0, 6 * len(df), 6)
    outside_co2 = 415
    val = 0
    df['new vals'] = None
    fix = df.iloc[:, 1].iloc[0]

    while True:

        for i in range(0,len(df)):
            v = outside_co2 - ((outside_co2 - fix)*math.exp(-val*df['time'].iloc[i]))

            df['new vals'].iloc[i] = v

        array1 = df.iloc[:, 1].values
        array2 = df['new vals'].values
        Y = np.sum((array1 - array2) ** 2)        
        
        if X == 0:
            X = Y
            val = val + 0.00001
        elif X > Y:
            val = val + 0.00001
            X = Y
        else:
            break
    return val * 60


In [5]:
df = pd.read_csv('C:\\Users\\vrana7\\Desktop\\CIA\Data Visualization\\data\\Mod-PM-06-29-2023.csv')
print(df)
columns_to_interpolate = ['pm1','pm25','pm10']
df[columns_to_interpolate] = df[columns_to_interpolate].interpolate(method='linear')
print(df)
df.to_csv('C:\\Users\\vrana7\\Desktop\\CIA\Data Visualization\\data\\Mod-PM-06-29-2023-edited.csv',index=False) 

                timestamp       timestamp_local  sample_rh  sample_temp  \
0    2023-06-29T21:20:34Z  2023-06-29T14:20:34Z      27.20        26.89   
1    2023-06-29T21:19:34Z  2023-06-29T14:19:34Z      27.00        26.89   
2    2023-06-29T21:18:34Z  2023-06-29T14:18:34Z      26.93        26.89   
3    2023-06-29T21:17:34Z  2023-06-29T14:17:34Z      26.98        26.89   
4    2023-06-29T21:16:34Z  2023-06-29T14:16:34Z      27.15        26.89   
..                    ...                   ...        ...          ...   
211  2023-06-29T17:49:34Z  2023-06-29T10:49:34Z      28.16        26.60   
212  2023-06-29T17:48:34Z  2023-06-29T10:48:34Z      28.05        26.59   
213  2023-06-29T17:47:34Z  2023-06-29T10:47:34Z      28.08        26.58   
214  2023-06-29T17:46:34Z  2023-06-29T10:46:34Z      28.15        26.57   
215  2023-06-29T17:45:34Z  2023-06-29T10:45:34Z      28.35        26.56   

       pm1   pm25    pm10  
0    3.820  4.064   4.294  
1    4.470  4.697   4.754  
2    4.553  4.7